## Dependencies

In [ ]:
!pip install --pre -I "weaviate-client==4.*"

## Configuration

In [ ]:
import weaviate, json, os

openAIKey = os.getenv("OPENAI_API_KEY")

client = weaviate.connect_to_embedded(
    version="1.22.3",
    headers={
        # "X-OpenAI-Api-Key": "OPENAI-API-KEY", # Replace with your OpenAI key
        "X-OpenAI-Api-Key": openAIKey, # Replace with your OpenAI key
    },
    
)

client.collections.list_all()  # Get the schema to test connection

## Create a new Collection

In [18]:
import weaviate.classes as wvc
# resetting the schema. CAUTION: THIS WILL DELETE JeopardyQuestion DATA
if client.collections.exists("JeopardyQuestion"):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(
        model="ada",
        model_version="002",
        type_="text"
    )
)

print("Successfully created the schema.")

Successfully created the schema.


{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"jeopardyquestion_rJk2Wy7ElnEo","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2023-11-08T16:28:08-08:00","took":45041}


## Import the Data

In [19]:
import requests
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text) # get source data

print(data)

[{'Category': 'SCIENCE', 'Question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'Answer': 'Liver'}, {'Category': 'ANIMALS', 'Question': "It's the only living mammal in the order Proboseidea", 'Answer': 'Elephant'}, {'Category': 'ANIMALS', 'Question': 'The gavial looks very much like a crocodile except for this bodily feature', 'Answer': 'the nose or snout'}, {'Category': 'ANIMALS', 'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'Answer': 'Antelope'}, {'Category': 'ANIMALS', 'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'Answer': 'the diamondback rattler'}, {'Category': 'SCIENCE', 'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'Answer': 'species'}, {'Category': 'SCIENCE', 'Question': 'A metal that is "ductile" can be pulled into this while cold & under pressure', 'Answer': 'wire'}, {'Categ

In [20]:
# straight insert - take #1
jeopardy = client.collections.get("JeopardyQuestion")
jeopardy.data.insert_many(data)

BatchObjectReturn(all_responses=[UUID('1bd53e75-5ee1-4ce0-a690-5aafc54bb8ba'), UUID('4c1ea14d-1a67-4b0f-a956-3d4f4ec21a27'), UUID('eec3f3ae-0a6c-4d92-b579-cbea2f609e43'), UUID('d90fbd0d-90ee-43d3-b1a9-ad8e5bd49347'), UUID('6c439342-59ea-483e-8fdc-5bbcaf395761'), UUID('0d952c9c-d422-4ef3-9b9c-1f23e51d9958'), UUID('84c55b83-9e08-45d6-bcc9-3163719481b9'), UUID('a602fe3f-159e-4cb0-a25c-247fa766dd20'), UUID('15603802-f426-47dd-afe6-294604934829'), UUID('55975cbe-2e67-4888-bcac-dded2ffee88f')], uuids={0: UUID('1bd53e75-5ee1-4ce0-a690-5aafc54bb8ba'), 1: UUID('4c1ea14d-1a67-4b0f-a956-3d4f4ec21a27'), 2: UUID('eec3f3ae-0a6c-4d92-b579-cbea2f609e43'), 3: UUID('d90fbd0d-90ee-43d3-b1a9-ad8e5bd49347'), 4: UUID('6c439342-59ea-483e-8fdc-5bbcaf395761'), 5: UUID('0d952c9c-d422-4ef3-9b9c-1f23e51d9958'), 6: UUID('84c55b83-9e08-45d6-bcc9-3163719481b9'), 7: UUID('a602fe3f-159e-4cb0-a25c-247fa766dd20'), 8: UUID('15603802-f426-47dd-afe6-294604934829'), 9: UUID('55975cbe-2e67-4888-bcac-dded2ffee88f')}, errors={

## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [nearText](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#neartext)

2. [nearObject](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#nearobject)

3. [nearVector](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#nearvector)

### nearText Example

Find a `JeopardyQuestion` about "animals in movies". Limit it to only 2 responses and report the distance.

In [21]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(query="question about animals", limit=2)

print(response.objects)
print(response.objects[0].metadata.uuid)

[_Object(properties={'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}, metadata=_MetadataReturn(uuid=UUID('4c1ea14d-1a67-4b0f-a956-3d4f4ec21a27'), vector=None, creation_time_unix=1699489693694, last_update_time_unix=1699489693694, distance=0.1744983196258545, certainty=0.9127508401870728, score=0.0, explain_score='', is_consistent=False)), _Object(properties={'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}, metadata=_MetadataReturn(uuid=UUID('eec3f3ae-0a6c-4d92-b579-cbea2f609e43'), vector=None, creation_time_unix=1699489693694, last_update_time_unix=1699489693694, distance=0.18087047338485718, certainty=0.909564733505249, score=0.0, explain_score='', is_consistent=False))]
4c1ea14d-1a67-4b0f-a956-3d4f4ec21a27


### nearObject Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to id `4c1ea14d-1a67-4b0f-a956-3d4f4ec21a27`. (The id was taken from the query above)

In [28]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_object(
    near_object="4c1ea14d-1a67-4b0f-a956-3d4f4ec21a27", # make sure to get it from the previous query
    limit=4,
)

for o in response.objects:
    print(o.properties)

{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}
{'answer': 'species', 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'}


### nearVector Example

Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector `[-0.0125526935, -0.021168863, ... ]`

In [32]:
# update this with your vector
q_vector = [0.0026038211761042476, -0.031377524361014366, -0.006337505161762238, -0.040944311767816544, -0.004996422212570906, 0.029025869444012642, -0.01155359297990799, -0.007938872091472149, -0.015877744182944298, -0.0019746232535690069, 0.03283652663230896, 0.004597709842026234, 0.024675090089440346, -0.010019870474934578, -0.013735937885940075, 0.01014148723334074, 0.033593652993440628, 0.027728622779250145, 0.01791820302605629, 0.006992964074015617, 0.0003441585577093065, -0.007905090227723122, -0.0014881163369864225, -0.026972295939922333, 0.04137672856450081, -0.022701796144247055, 0.008817215450108051, -0.012519770301878452, -0.015580459497869015, -0.007905090227723122, 0.0083036824837327, -0.007722664624452591, -0.011330628767609596, -0.03216087818145752, -0.009350977838039398, -0.024607525430619717, 0.007628073915839195, -0.016148004680871964, 0.03380946069955826, -0.006219346541911364, 0.006020030006766319, 0.010148243978619576, -0.0161074660718441, -0.004219426307827234, -0.02495846338570118, 0.0029120459221303463, -0.0025455064605921507, -0.022715309634804726, -0.008932076394557953, 0.009459082037210464, 0.026674611493945122, 0.003146833973005414, -0.010573902167379856, -0.0041484832763671875, -0.007236197590827942, -0.025417905300855637, -0.019391117617487907, -0.008128054440021515, 0.007310518994927406, -0.012911646626889706, 0.00036611713585443795, 0.01660744473338127, -0.016702035441994667, 0.016999321058392525, 0.008526686578989029, -0.0029475174378603697, 0.0002812387829180807, -0.0001646893797442317, 0.013479191809892654, 0.01089821383357048, 0.050215010046958923, 0.00804021954536438, -0.030215010046958923, -0.014377804473042488, 0.014323752373456955, -0.018742496147751808, -0.00557410204783082, 0.00904693640768528, 0.01640474982559681, -0.0019323553424328566, 0.023499062284827232, -0.015418303199112415, -0.01929652690887451, 0.008425339125096798, 0.03205277398228645, -0.01760740578174591, 0.007391597144305706, 0.010864431038498878, -0.004097809549421072, 0.017350658774375916, 0.01626962050795555, 0.006739595904946327, 0.004040379077196121, 0.02278287522494793, -0.009709072299301624, 0.004165374208241701, 0.01258057914674282, -0.0006405994645319879, 0.026512455195188522, -0.021566707640886307, -0.020715389400720596, 0.007357814349234104, -0.0030674452427774668, -0.005959221627563238, -0.0016730751376599073, -0.00406064884737134, -0.012654900550842285, 0.026769202202558517, 0.015093992464244366, -0.008202374912798405, -0.01793171651661396, 0.05213305354118347, 0.021147804334759712, -0.04459281638264656, 0.004888318479061127, -0.029620438814163208, 0.028052935376763344, 0.023701757192611694, -0.018485749140381813, 0.002200925722718239, 0.024931438267230988, -0.006624735426157713, 0.01266165729612112, -0.026093553751707077, 0.005851117894053459, 0.008019950240850449, -0.022363971918821335, -0.04243073984980583, 0.007229441311210394, -0.010249591432511806, 0.03113389201462269, 0.00019836623687297106, -0.003841063007712364, 0.01315488014370203, 0.012553553096950054, 0.010877944529056549, -0.010709032416343689, -0.0014847380807623267, 0.019837046042084694, -0.01724255457520485, 0.023228803649544716, 0.021742375567555428, 0.0005768350674770772, -0.0024897654075175524, -0.029620438814163208, 0.030052855610847473, 0.04186319559812546, 0.03113389201462269, -0.00787130743265152, 0.018310079351067543, 0.02579626813530922, -0.0027194861322641373, 0.026215169578790665, 0.0022684906143695116, 0.021985609084367752, -0.00048266653902828693, -0.011837365105748177, 0.010134730488061905, -0.019188422709703445, -0.02460712566971779, 0.0067598652094602585, -8.387545676669106e-05, 0.018728982657194138, -0.0008061333792284131, 0.01778307370841503, 0.0462413989007473, 0.014742654748260975, 0.02749890275299549, 0.029106946662068367, -0.0030353518668562174, -0.02848535031080246, 0.01306704618036747, -0.012871108017861843, 0.020864032208919525, 0.006104486063122749, 0.02579626813530922, -0.0036485029850155115, 0.02213425189256668, -0.003841063007712364, -0.017985768616199493, -0.005604506004601717, 0.0048511577770113945, 0.008634790778160095, 0.008702355436980724, -0.008499660529196262, -0.00870911218225956, 0.0058477395214140415, -0.016012875363230705, 0.00921584852039814, -0.0069456687197089195, 0.01674257591366768, 0.014810219407081604, -0.01606692560017109, 0.007715908344835043, -0.6360827088356018, -0.04761972278356552, 0.022931518033146858, -0.0077429343946278095, 0.013438653200864792, 0.021661298349499702, 0.003060688730329275, 0.0024407808668911457, -0.009742854163050652, 0.026539482176303864, 0.014012954197824001, -0.011817095801234245, -0.0050437175668776035, -0.012148164212703705, -0.00822264514863491, -0.007351058069616556, 0.0269854087382555, -0.015661537647247314, -0.004601167514920235, -0.011958981864154339, -0.02312069945037365, 0.015296686440706253, 0.012742734514176846, 0.02530980110168457, 0.016283133998513222, 0.008783433586359024, 0.03659313544631004, -0.03102578967809677, -0.0045572505332529545, 0.03348515182733536, -0.036917444318532944, 0.02845832332968712, -0.0015911527443677187, -0.004026866517961025, 0.05048447102308273, -0.012621117755770683, -0.009661776944994926, 0.032755449414253235, 0.0011029965244233608, 0.009026667103171349, -0.025350339710712433, -0.02013433165848255, -0.00019646597502287477, -0.015715589746832848, 0.011350898072123528, -0.0019053294090554118, 0.02562059834599495, -0.0002810276346281171, 0.010188782587647438, -0.006992964074015617, 0.006077460013329983, 0.015066966414451599, -0.029079919680953026, 0.008087514899671078, -0.021080240607261658, 0.0061416467651724815, 0.022674771025776863, -0.025026028975844383, 0.007344301324337721, 0.00339682400226593, 0.017229042947292328, -0.01559397205710411, 0.010073922574520111, -0.00854695588350296, -0.047998085618019104, 0.015688562765717506, -0.016891218721866608, 0.006888238713145256, 0.020850518718361855, -0.027350259944796562, 0.027917804196476936, -0.004695758689194918, -0.02313421107828617, 0.003847819520160556, 0.018215488642454147, 0.02764754556119442, 0.025593573227524757, -0.01096577849239111, -0.01740471087396145, 0.015715589746832848, -0.0011224214686080813, 0.0034998604096472263, -0.01247923169285059, -0.012445448897778988, 0.018850598484277725, -0.006705813575536013, -0.014526447281241417, -0.008594251237809658, 0.0035403992515057325, 0.008992884308099747, 0.04272802546620369, 0.022836927324533463, 0.0033850001636892557, 0.014067006297409534, 0.012573822401463985, -0.034593213349580765, -0.003513373201712966, -0.01559397205710411, 0.016837166622281075, -0.03872818499803543, 0.006908508017659187, 0.0022532884031534195, 0.021728862076997757, -0.0013681886484846473, 0.009445568546652794, 0.002204303862527013, -0.03305273503065109, 0.03432295471429825, 0.032431136816740036, -0.015472355298697948, -0.01991812326014042, -0.006813917309045792, 0.00586800929158926, 0.010972535237669945, 0.00033972461824305356, -0.02797185629606247, 0.024890897795557976, 0.008465878665447235, 0.006401771679520607, -0.019999202340841293, 0.02144508995115757, 0.018728982657194138, 0.00494574848562479, -0.005148443393409252, 0.02009379304945469, 0.009175309911370277, 0.0012879553250968456, -0.026363812386989594, -0.016161516308784485, -0.0018563448684290051, 0.00711458083242178, -0.016148004680871964, 0.010857675224542618, 0.014472395181655884, 0.018972216174006462, 0.01810738630592823, 0.032079800963401794, -0.00031966628739610314, 0.0122427549213171, 0.01909383200109005, -0.007601047866046429, -0.013357575051486492, -0.002243153750896454, -0.015066966414451599, -0.02162075787782669, -0.03434998169541359, -0.02145860344171524, 0.006756487302482128, -0.03151225671172142, -0.008438852615654469, -0.009526646696031094, -0.029404232278466225, -0.008384800516068935, 0.02095862291753292, 0.021539680659770966, -0.01039823330938816, -0.022539641708135605, 0.012215728871524334, -0.0021654542069882154, -0.003567425301298499, -0.0017212150851264596, 0.010046896524727345, -0.033755410462617874, -0.0009053692920133471, -0.0044491467997431755, -0.0331878662109375, 0.00536465086042881, -0.0034762127324938774, -0.03270139917731285, -0.05324111878871918, -0.010357694700360298, -0.007391597144305706, 0.014621037989854813, 0.03202575072646141, 0.006965938024222851, 0.00039293195004574955, -0.004411986097693443, -0.016458801925182343, -0.016539881005883217, -0.0032025747932493687, 0.013796746730804443, -0.012100868858397007, 0.0014678467996418476, -0.004435633774846792, 0.030539322644472122, -0.024755768477916718, 0.041484832763671875, 0.03099876269698143, -0.03659313544631004, 0.03943086043000221, 0.012263024225831032, 0.04086323454976082, 0.0073645710945129395, -0.017877664417028427, 0.0031789271160960197, -0.007607804611325264, 0.0289988424628973, 0.03126902133226395, 0.019161397591233253, 0.027579979971051216, 0.006604466121643782, -0.009114501066505909, 0.01726958155632019, -0.010425259359180927, 0.010709032416343689, -0.0167560875415802, -0.007986167445778847, -0.044160399585962296, 0.012107624672353268, 0.027025949209928513, 0.02716107852756977, -0.008945588953793049, -0.0121616767719388, -0.020242435857653618, 0.0018090493977069855, 0.027093512937426567, 0.00591192627325654, 0.0072564673610031605, 0.00822264514863491, 0.0032515593338757753, 0.004043757449835539, -0.016526367515325546, 0.011283333413302898, 0.010148243978619576, 0.009310439229011536, 0.024863872677087784, 0.010904970578849316, 0.030025828629732132, -0.018364131450653076, -0.02226938121020794, -0.0035775599535554647, 0.015053452923893929, 0.002239775378257036, -0.0030674452427774668, 0.0031164297834038734, -0.03434998169541359, 0.00203032442368567, -0.016675010323524475, 0.028620479628443718, -0.011776557192206383, -0.01979650743305683, 0.023701757192611694, 0.022634232416749, -0.00805373303592205, 0.014607525430619717, 0.012594091705977917, -0.0027600249741226435, 0.02010730654001236, -0.009080718271434307, 0.022634232416749, 0.002305651316419244, -0.01483724545687437, -0.030755529180169106, 0.012492744252085686, 0.027012435719370842, -0.013749451376497746, 0.01862087845802307, 0.025026028975844383, 0.011344142258167267, 0.027066487818956375, 0.008905050344765186, 0.015296686440706253, -9.036801202455536e-05, -0.00464846333488822, 0.013472435064613819, -0.017053373157978058, 0.005009935237467289, -0.01960732601583004, 0.022863952443003654, -0.0036349899601191282, -0.01744524948298931, 0.0037498504389077425, -0.016377724707126617, -0.014094032347202301, 0.02129644714295864, -0.00629028957337141, -0.013634591363370419, 0.01944516971707344, 0.003045486519113183, -0.007074042223393917, -0.027025949209928513, -0.031755488365888596, 0.013614321127533913, 0.024836847558617592, -0.018553312867879868, -0.016012875363230705, -0.005094391293823719, 0.006641626823693514, -0.02566113881766796, 0.04407932236790657, 0.00995906163007021, 0.019228963181376457, -0.0029897454660385847, 0.007918602786958218, 0.007722664624452591, -0.036971498280763626, 0.009884740225970745, 0.003989705815911293, 0.03405269607901573, -0.012033303268253803, 0.004202534910291433, 0.003043797565624118, -0.011296846903860569, 0.00034141374635510147, 0.06361908465623856, -0.0027194861322641373, -0.009432055987417698, -0.040998365730047226, 0.011979252099990845, 4.8509467887924984e-05, -0.008465878665447235, -0.026742175221443176, 0.0016654740320518613, -0.024890897795557976, -0.010668492875993252, 0.001162115833722055, -0.006263263523578644, -0.017485788092017174, 0.02813401259481907, 0.015269661322236061, -0.009269900619983673, -0.04002542793750763, -0.013688642531633377, -0.007857794873416424, 0.07632128149271011, 0.013242715038359165, -0.015269661322236061, 0.010871187783777714, -0.009682046249508858, 0.008661816827952862, -0.015823692083358765, -0.02935018017888069, 0.029539361596107483, -0.009155039675533772, -0.001819184166379273, 0.004719406366348267, 0.015364252030849457, 0.009844201616942883, 0.02197209559381008, 0.008033462800085545, -0.02599896304309368, -0.017323633655905724, 0.03389053791761398, 0.011181985959410667, 0.002459361217916012, 0.011986007913947105, 0.021904531866312027, 0.014931836165487766, -0.004067405126988888, 0.003043797565624118, 0.009438812732696533, 0.01962083950638771, 0.010911726392805576, -0.008742894046008587, -0.01039823330938816, -0.01991812326014042, -0.023863911628723145, 0.009492864832282066, 0.0005016691866330802, 0.011864391155540943, 0.007864550687372684, 0.011533323675394058, 0.03840387240052223, 0.004685624036937952, 0.022918004542589188, 0.021755889058113098, 0.0029779216274619102, -0.017310120165348053, -0.001329338876530528, 0.017526326701045036, -0.010492824949324131, 0.0271475650370121, 0.016120977699756622, 0.0020607286132872105, 0.03380946069955826, 0.007925359532237053, -0.012080598622560501, -0.0034508758690208197, 0.014702116139233112, 0.00460792426019907, -0.021039700135588646, 0.0001994219346670434, -0.006952425464987755, -0.019147884100675583, -0.0288366861641407, -0.003060688730329275, -0.012803543359041214, 0.0049592615105211735, 0.003759985091164708, -0.014864271506667137, -0.032917603850364685, 0.0005561433499678969, -0.049187228083610535, -0.006253128871321678, -0.009357734583318233, -0.014877784997224808, -0.02193155698478222, -0.015296686440706253, 0.025877345353364944, -0.004141726531088352, 0.022647744044661522, -0.014931836165487766, 0.023742295801639557, 0.009729341603815556, 0.014107544906437397, -0.004918722435832024, -0.015066966414451599, -0.025390878319740295, 4.191133484710008e-05, 0.011127933859825134, 0.019404631108045578, -0.018769521266222, -0.005243034102022648, 0.0006030164659023285, -0.006148403510451317, -0.001418862259015441, 0.007290249690413475, 0.003655259497463703, 0.017134452238678932, -0.028944790363311768, -0.00427347794175148, 0.027701597660779953, -0.018012795597314835, -0.012877864763140678, 0.03524183854460716, -0.013931876048445702, 0.0072564673610031605, -0.0003758296079467982, 0.005124795716255903, 0.000375407311366871, -0.0013242714339867234, -0.02110726572573185, -0.024863872677087784, -0.009337465278804302, 0.03756606951355934, 0.0017837126506492496, 0.020850518718361855, -0.00812129769474268, 0.016823653131723404, 0.0019137749914079905, 0.003996462095528841, 0.01693175733089447, 0.014526447281241417, -0.015148044563829899, -0.006874725688248873, -0.013080558739602566, 0.028944790363311768, -0.0007465918315574527, -0.010850918479263783, 0.02480982057750225, -0.001000804710201919, 0.016026386991143227, -0.0010962400119751692, -0.0034086478408426046, 0.017485788092017174, 0.04394419118762016, 0.005915304645895958, -0.020674850791692734, -0.02848535031080246, 0.002342812018468976, -0.008634790778160095, 0.007918602786958218, -0.007938872091472149, -0.004783592652529478, 0.0002130404900526628, -0.02763403207063675, 0.009195579215884209, -0.00787130743265152, 0.013060289435088634, -0.028728583827614784, -0.01449942123144865, 0.024053094908595085, -0.009290169924497604, -0.017877664417028427, -0.029052894562482834, -0.009729341603815556, -0.0046349503099918365, 0.0013690332416445017, -0.011506297625601292, -0.036106668412685394, -0.018215488642454147, 0.013688642531633377, 0.03035013936460018, 0.03653908148407936, 0.007330788299441338, 0.0020032983738929033, 0.015134531073272228, 0.018810059875249863, 0.0030285953544080257, 0.0009780015097931027, -0.00912801455706358, -0.009688802994787693, -0.02214776538312435, 0.017337145283818245, 0.02376932092010975, -0.028404271230101585, -0.0019813398830592632, 0.006918642669916153, 0.016877705231308937, 0.03521481156349182, -0.0028630613815039396, -0.026269221678376198, -0.025269262492656708, -0.01422916166484356, -0.013344062492251396, 0.005550454370677471, 0.004462659824639559, 0.00019667712331283838, -0.022363971918821335, -0.015188583172857761, 0.02412065863609314, 0.018674930557608604, -0.016675010323524475, -0.006999720819294453, 0.01331703644245863, -0.01909383200109005, 0.013256227597594261, 0.00997257512062788, 0.002410376677289605, -0.01924247480928898, -0.015215609222650528, 0.020850518718361855, -0.00795238558202982, -0.006709191482514143, 0.003891736501827836, 0.012283293530344963, 0.01047931145876646, 0.026025988161563873, -0.01499940175563097, 0.012884620577096939, -0.018350619822740555, -0.022080199792981148, 0.022647744044661522, -0.02729620784521103, -0.01759389229118824, -0.011465759016573429, -0.02463415265083313, -0.03521481156349182, -0.0028225225396454334, 0.017296606674790382, -0.013344062492251396, 0.011296846903860569, 0.001754997530952096, -0.03016095794737339, 0.00427010003477335, 0.001573416986502707, 0.021026188507676125, -0.004216047935187817, 0.022377485409379005, -0.0006101952167227864, -0.0169182438403368, -0.019715430215001106, 0.014350778423249722, 0.03437700495123863, 0.015377764590084553, 0.009276656433939934, 0.02147211693227291, -0.00797265488654375, -0.016323672607541084, 0.016877705231308937, 0.0005067365127615631, -0.03035013936460018, -0.023039620369672775, 0.028863713145256042, -0.010094191879034042, 0.012276536785066128, 0.0003445808542892337, -0.02059377357363701, -0.021161317825317383, 0.01199276465922594, -0.009405029937624931, 0.006185563746839762, -0.010992804542183876, -0.038782235234975815, 0.0023715270217508078, 0.02528277412056923, -0.023566627874970436, -0.007709151599556208, -0.01121576875448227, -0.013006237335503101, -0.010790109634399414, -0.005857874173671007, 0.004009975120425224, -0.008891536854207516, -0.0017921582330018282, 0.011350898072123528, -0.013310279697179794, 0.0031333209481090307, 0.02312069945037365, -0.01283056940883398, -0.013033263385295868, -0.009580698795616627, -0.004158617928624153, 0.030944710597395897, -0.008938832208514214, -0.0005772573640570045, 0.010161756537854671, 0.023823373019695282, -0.017053373157978058, -0.018647903576493263, -0.010844161733984947, -0.005107904318720102, -0.022688284516334534, -0.0018343862611800432, 0.02094510942697525, 0.022188303992152214, -0.01315488014370203, -0.004422120749950409, -0.012864351272583008, -0.0019796506967395544, -0.03632287308573723, -0.00018337529036216438, -0.03262031823396683, 0.0007639053510501981, -0.020904570817947388, -0.004999800585210323, 0.019215449690818787, 0.005300464108586311, 0.0069389124400913715, -0.009999601170420647, -0.01331703644245863, 0.021499142050743103, -0.005945708602666855, 0.005327490158379078, -0.024917924776673317, 0.0027346881106495857, -0.03356622904539108, -0.004699136596173048, -0.006972694769501686, -0.007830768823623657, 0.008621277287602425, -0.011749531142413616, -0.020323513075709343, -0.01775604858994484, 0.010121217928826809, 0.025526007637381554, -0.015296686440706253, -0.016364211216568947, 0.02595842443406582, 0.021336985751986504, 0.024580100551247597, -0.029079919680953026, -0.035458043217659, 0.022350460290908813, 0.012276536785066128, -0.02663407288491726, 0.017485788092017174, -0.017337145283818245, -0.0031400774605572224, 0.011330628767609596, -0.00036104978062212467, -0.008141566999256611, -0.040133532136678696, -0.02851237542927265, -0.015877744182944298, 0.026053015142679214, -0.009479351341724396, -0.0167560875415802, -0.024566587060689926, -0.02964746579527855, -0.0012001210125163198, 0.0024661177303642035, -0.010283373296260834, -0.021904531866312027, -0.00011749953409889713, 0.03616071864962578, 0.004675488919019699, 0.004830888472497463, 6.524232594529167e-05, -0.033917564898729324, -0.0017448627622798085, -0.026944870129227638, -0.035133734345436096, -0.005503158550709486, -0.024850359186530113, 0.03229600936174393, -0.012114381417632103, -0.004476172849535942, -0.045160360634326935, 0.023553114384412766, -0.03683636710047722, -0.0011857635108754039, -0.004107944201678038, 0.021715350449085236, 0.017296606674790382, 0.006060569081455469, 0.004631571937352419, 0.022999081760644913, -0.004290369339287281, 0.019526246935129166, -0.00929692666977644, -0.01590477116405964, -0.013877824880182743, -0.02498548850417137, -0.01004013977944851, 0.005614640656858683, -0.02848535031080246, -0.006155159790068865, 0.034106746315956116, 0.0011680277530103922, 0.0009239496430382133, 0.010154999792575836, -0.04372798651456833, -0.014188623055815697, -0.020364051684737206, -5.795799006591551e-05, 0.024999001994729042, 0.0010818825103342533, -0.004509955178946257, -0.015445329248905182, -0.015620998106896877, 0.0026502320542931557, -0.01272922195494175, -0.008168593049049377, -0.033917564898729324, -0.008067245595157146, -0.0027617141604423523, 0.027674570679664612, -0.023201776668429375, 0.006827430333942175, -0.0020320136100053787, 0.004209291655570269, 0.013215688988566399, -0.004033622797578573, -0.010513094253838062, 0.027890779078006744, -0.006817295681685209, -0.02498548850417137, -0.027228642255067825, 0.01056038960814476, -0.012587335892021656, 0.0032904092222452164, 0.012040060013532639, -0.015269661322236061, 0.009905010461807251, -0.0103103993460536, 0.002498211106285453, -0.03651205822825432, -0.008026706986129284, -0.0038343064952641726, 0.003702554851770401, -0.03991732746362686, 0.0362417958676815, 0.006053812336176634, -0.004121457226574421, -0.04426850378513336, -0.017012834548950195, -0.006621357519179583, 0.006297045852988958, -0.02851237542927265, -0.015310199931263924, -0.0167560875415802, -0.01004013977944851, 0.010945509187877178, -0.018634391948580742, -0.041619960218667984, -0.012208972126245499, -0.013958902098238468, -0.033944591879844666, 0.01155359297990799, 0.1649663895368576, -0.008844241499900818, 0.03429592773318291, 0.01693175733089447, 0.0022769360803067684, 0.032106827944517136, 0.03199872374534607, -0.008033462800085545, -0.005303842481225729, -0.002851237542927265, 0.003912006039172411, 0.00452346820384264, -0.016202056780457497, 0.009378003887832165, -0.0032380465418100357, -0.0236071664839983, -0.032728422433137894, 0.02428281493484974, -0.0063578542321920395, -0.01875600777566433, 0.013722425326704979, -0.016337186098098755, -0.008452365174889565, 0.005374785512685776, 0.01199276465922594, 0.007999680936336517, 0.009979331865906715, -0.010763083584606647, 0.007830768823623657, -0.008756407536566257, -0.015445329248905182, -0.009567185305058956, -0.0005025137215852737, 0.018039820715785027, -0.038457926362752914, 0.0034137151669710875, -0.02280990034341812, 0.00972258485853672, 0.010790109634399414, 0.004611302632838488, 0.011533323675394058, -0.0075334832072257996, 0.018634391948580742, 0.020674850791692734, -0.02663407288491726, 0.013458922505378723, -0.010600928217172623, -0.015553433448076248, -0.014891297556459904, 0.02326934225857258, -0.02330988086760044, 0.030620399862527847, 0.021053213626146317, 0.019823532551527023, 0.0009940481977537274, -0.022836927324533463, -0.006418662611395121, -0.0020641067530959845, -0.002388418186455965, -0.00012837325630243868, -0.013357575051486492, 0.02126942202448845, 0.009060448966920376, 0.030242037028074265, -0.009337465278804302, 0.007817255333065987, -0.0004526846460066736, 0.024904411286115646, 0.0020421482622623444, 0.011783313937485218, -0.01960732601583004, -0.023539600893855095, -0.025080081075429916, -0.00548964599147439, -0.019972175359725952, -0.024593614041805267, 0.04305233433842659, 0.007236197590827942, 0.019499221816658974, -0.00903342291712761, -0.02664758451282978, -0.010972535237669945, -0.025674650445580482, -0.005851117894053459, -0.0047667017206549644, -0.034079719334840775, 0.019837046042084694, 0.023823373019695282, -0.008898293599486351, 0.01929652690887451, -0.020850518718361855, -0.012377884238958359, -0.009256387129426003, 0.010080678388476372, 0.004158617928624153, 0.0084591219201684, -0.021728862076997757, 0.03197169676423073, -0.019066806882619858, -0.03099876269698143, -0.023512575775384903, 0.019228963181376457, 0.00549302389845252, -0.015026427805423737, -0.009986087679862976, 0.0025843563489615917, -0.003918762318789959, 0.022228842601180077, 0.00732403201982379, -0.02095862291753292, -0.022918004542589188, -0.016877705231308937, -0.0005975268431939185, -0.014594011940062046, -0.0046417065896093845, 0.016323672607541084, -0.010722544975578785, -0.013533243909478188, 0.018174950033426285, -0.0030944710597395897, -0.005003178492188454, -0.03983624652028084, 0.0006929622031748295, 0.02749890275299549, 0.005087635014206171, -0.03424187749624252, -0.013303522951900959, 0.04424147680401802, -0.0035032385494560003, -0.010100948624312878, -0.011526566930115223, -0.0038140369579195976, -0.003743093926459551, -0.006871347315609455, -0.0032616942189633846, 0.02430984005331993, -0.00033296813489869237, -0.006901751738041639, -0.015850719064474106, 0.0063578542321920395, 0.00011032076872652397, 0.011080638505518436, 0.0060808383859694, 0.022904491052031517, 0.012904890812933445, -0.01239815354347229, 0.029755569994449615, -7.426857337122783e-05, -0.006533523090183735, -0.019026268273591995, -0.023715270683169365, -0.00544910691678524, 0.010864431038498878, -0.004310638643801212, 0.04772782325744629, -0.008033462800085545, -0.01483724545687437, -0.04645760729908943, -0.01690473034977913, 0.012425179593265057, -0.011425219476222992, 0.016161516308784485, 0.01862087845802307, -0.03856602683663368, -0.010188782587647438, 0.013675129972398281, -0.1721012443304062, 0.003107984084635973, 0.02964746579527855, -0.004009975120425224, 0.0033951348159462214, -0.0007736177649348974, 0.009378003887832165, 0.001672230544500053, -0.029728543013334274, 0.003405269468203187, 0.009330708533525467, -0.008114540949463844, -0.048511575907468796, -0.01659393310546875, -0.003854575799778104, -0.025701677426695824, 0.018837086856365204, -0.005378163885325193, 0.012546796351671219, 0.015134531073272228, -0.013506217859685421, -0.02276936173439026, -0.016377724707126617, 0.013019750826060772, -0.01828305423259735, 0.00856046937406063, 0.0003038307768292725, 0.01978299394249916, 0.018877625465393066, -0.03551209717988968, -0.00012826768215745687, 0.011911686509847641, 0.03083660639822483, 0.015053452923893929, -0.0008344261441379786, 0.015864232555031776, -0.0017271270044147968, -0.0023140967823565006, 0.008182105608284473, 0.018837086856365204, -0.001016851281747222, 0.02799888327717781, 0.018877625465393066, -0.009938792325556278, 0.025512496009469032, 0.01776956021785736, -0.004023488145321608, 0.01809387281537056, 0.015269661322236061, -0.007175389211624861, 0.02330988086760044, -0.03113389201462269, 0.012087355367839336, 0.009053693152964115, 0.017674969509243965, 0.0010734369279816747, 0.011222525499761105, 0.01626962050795555, -0.002157008508220315, -0.027039460837841034, -0.016120977699756622, -0.0478629544377327, 0.027039460837841034, -0.0008893226040527225, -0.01609395258128643, -0.013452165760099888, -0.029782595112919807, 0.018391158431768417, -0.020391078665852547, -0.0015075411647558212, -0.017039859667420387, 0.00041805763612501323, 0.015377764590084553, -0.02276936173439026, 0.005797065794467926, -0.002347879344597459, -0.003486347384750843, 0.005313977133482695, -0.046160321682691574, -0.005415324587374926, -0.017350658774375916, 0.021147804334759712, -0.01991812326014042, -0.013702156022191048, 0.022863952443003654, -0.005297085735946894, -0.010445528663694859, -0.013425139710307121, 0.0018529666122049093, -0.003550533903762698, 0.032431136816740036, -0.04610626772046089, -0.012377884238958359, 0.014891297556459904, 0.02832319401204586, 0.03472834452986717, -0.019310040399432182, 0.004972774535417557, -0.0035201297141611576, 0.004780214745551348, 0.001991474535316229, 0.0011697168229147792, -0.027404312044382095, 0.018229002133011818, 0.024539561942219734, 0.0023529466707259417, 0.027539441362023354, -0.0009889807552099228, 0.04605221748352051, 0.018310079351067543, 0.001795536489225924, 0.01609395258128643, 0.017337145283818245, 0.04159293323755264, 0.007006477098912001, 0.009749610908329487, 0.005986247677356005, -0.036620158702135086, 0.030917685478925705, 0.017147963866591454, 0.03240411356091499, -0.017472276464104652, 0.00012182790669612586, -0.0012676859041675925, 0.0012001210125163198, -0.016310159116983414, -0.10415800660848618, -0.030377166345715523, 0.002138428157195449, 0.03289058059453964, -0.0093036824837327, 0.02964746579527855, -0.0009560429025441408, 0.031106866896152496, -0.0362417958676815, 0.01724255457520485, -0.004351177718490362, -0.018458722159266472, 0.0020421482622623444, -0.00121532310731709, 0.022742334753274918, -0.019174911081790924, -0.02578275464475155, -0.01147251483052969, -0.01962083950638771, 0.010249591432511806, 0.012756248004734516, -0.025404391810297966, 0.013783234171569347, 0.0019475574372336268, 0.0048680491745471954, -0.009222605265676975, 0.002979610813781619, -0.01005365327000618, 0.013006237335503101, -0.006209211889654398, 0.015418303199112415, -0.023080160841345787, -0.0008546956232748926, -0.0004666198801714927, 0.013107584789395332, -0.009600968100130558, -0.017688482999801636, -0.02044513076543808, 0.04272802546620369, -0.025553034618496895, 0.005479510873556137, 0.0062497504986822605, -0.012722465209662914, -0.022377485409379005, -0.005499780643731356, -0.023688243702054024, -0.002847859403118491, 0.015188583172857761, 0.0007427912787534297, -0.030755529180169106, -0.021391037851572037, -0.015539919957518578, -0.03605261445045471, -0.0002328876726096496, 0.02412065863609314, -0.02548546902835369, 0.016715548932552338, 0.03667421266436577, -0.003001569304615259, 0.009999601170420647, 0.026417864486575127, -0.01641826331615448, 0.010661737062036991, 0.021364012733101845, 0.021904531866312027, 0.0322149321436882, -0.01706688664853573, -0.015783153474330902, 0.02498548850417137, -0.012513014487922192, 0.01794523000717163, 0.01081713568419218, -0.026728663593530655, -0.013526487164199352, -0.04040379449725151, 0.010323911905288696, -0.036782316863536835, -0.006884860340505838, 0.01408051885664463, -0.013181906193494797, -0.005161955952644348, -0.03856602683663368, -0.008013193495571613, -0.02398552931845188, 0.01096577849239111, 0.024593614041805267, 0.0020201895385980606, -0.014026467688381672, -0.005384920164942741, 0.015283173881471157, 0.010425259359180927, 0.023890938609838486, -0.0020978893153369427, -0.03434998169541359, -0.010249591432511806, 0.005563966929912567, 0.005435593891888857, 0.0007487032562494278, 0.035133734345436096, 0.006911886390298605, -0.0034170933067798615, -0.014485908672213554, -0.04375500977039337, 0.025012515485286713, -0.005016691517084837, -0.010107704438269138, 0.010350937955081463, -0.009438812732696533, 0.004577519837766886, -0.018864111974835396, 0.03232303261756897, 0.013877824880182743, -0.029377205297350883, 0.042322635650634766, -0.006347719579935074, -0.01458049938082695, -0.00881045963615179, -0.013891337439417839, 0.03821469098329544, 0.007776716724038124, 0.024512534961104393, 0.01806684583425522, -0.0019374226685613394, 0.020661337301135063, 0.013100828975439072, 0.02864750474691391, -0.013290010392665863, -0.0033461502753198147, -0.006881481967866421, 0.04134970158338547, -0.028377246111631393, 0.011073882691562176, 0.008952345699071884, -0.022647744044661522, -0.02510710619390011, 0.02597193606197834, 0.00030488648917526007, -0.007121337577700615, 0.005161955952644348, 0.010661737062036991, -0.009884740225970745, 0.02880966104567051, 0.0015835516387596726, -0.01081713568419218, -0.0032701396849006414, -0.023904452100396156, -0.007297005970031023, -0.0040640272200107574, -0.023188263177871704, 0.01925598829984665, 0.021999122574925423, -0.028431298211216927, 0.013134610839188099, 0.006550414487719536, -0.0014729142421856523, -0.004324151668697596, -0.01624259538948536, -0.01712093874812126, 0.029863672330975533, 0.028620479628443718, 0.00770239531993866, 0.0025944910012185574, 0.018391158431768417, 0.012790029868483543, 0.014485908672213554, 0.0008394935284741223, -0.008506417274475098, -0.005645045079290867, 0.006783512886613607, 0.005263303406536579, -0.012269780971109867, -0.02967449091374874, -0.019891098141670227, -0.002496521919965744, -0.004320773296058178, 0.005371407140046358, -0.01408051885664463, -0.00797265488654375, -0.0006929622031748295, 0.0030589995440095663, -0.025580059736967087, 0.022039661183953285, -0.0047430540435016155, 0.0036045857705175877, -0.008344261907041073, 0.010249591432511806, 0.028620479628443718, 0.008398313075304031, 0.00837128795683384, 0.006972694769501686, 5.267948654363863e-05, 0.0012085665948688984, -0.015148044563829899, 0.005915304645895958, -0.031106866896152496, 0.025539521127939224, 0.0026113821659237146, 0.0029204916208982468, -0.01006040908396244, -0.001988096395507455, 0.012019790709018707, 0.02564762532711029, -0.006763243582099676, 0.00196782685816288, -0.0023529466707259417, 0.001775266951881349, -0.025390878319740295, -0.004925479181110859, -0.0034593213349580765, -0.04005245491862297, -0.028269141912460327, 0.004969396162778139, 0.01281705591827631, 0.001459401217289269, -0.009763123467564583, 0.0027802945114672184, 0.0003302232944406569, 0.02378283441066742, 0.0006752264453098178, -0.004854536149650812, -0.03362027928233147, 0.024350380524992943, -0.002010054886341095, -0.008594251237809658, 0.013188662938773632, -0.004084296524524689, 0.017161477357149124, 0.006786891259253025, 0.026431377977132797, -0.007074042223393917, 0.02112077921628952, -0.01157386228442192, 0.00494574848562479, -0.01858033984899521, 0.002837724518030882, -0.035106707364320755, -0.020242435857653618, -0.006310558877885342, -0.0015683495439589024, 0.032917603850364685, -0.01724255457520485, 0.05697070062160492, -0.012783274054527283, 0.003273518057540059, 0.04402526840567589, -0.008573981933295727, 0.03194466978311539, 0.005759905092418194, 0.05197089910507202, 0.008296966552734375, 0.022390998899936676, 0.013107584789395332, -0.016526367515325546, 0.004577519837766886, -0.024066606536507607, -0.03337704762816429, 0.014810219407081604, 0.005520049948245287, 0.0059558432549238205, -0.008384800516068935, -0.024499021470546722, 0.026877306401729584, 0.022863952443003654, 0.028269141912460327, -0.007837524637579918, -0.008945588953793049, -0.005699096713215113, 0.01165494043380022, -0.007128093857318163, -0.010263103991746902, 0.001776956138201058, -0.012046816758811474, 0.0007537705823779106, -0.03302570804953575, -0.015864232555031776, -0.019647864624857903, -0.009438812732696533, -0.001132556120865047, 0.005320733413100243, 0.003955923020839691, -0.0093036824837327, -0.011506297625601292, 0.001922220573760569, -0.006641626823693514, -0.029917724430561066, 0.020350540056824684, -0.010080678388476372, 0.0012499501463025808, -0.024701716378331184, -0.043295569717884064]

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_vector(
    near_vector=q_vector,
    limit=2,
    return_properties=["question", "answer"],
    return_metadata=wvc.MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea"}
0.0004520416259765625
{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa'}
0.14741843938827515
